In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import re

pit_data = pd.read_csv("driver_race_pit_analysis_1994_2022.csv")

def extract_year(race_id):
    match = re.search(r'_(\d{4})$', str(race_id))
    return int(match.group(1)) if match else None

def extract_race_name(race_id):
    match = re.search(r'^(.+)_\d{4}$', str(race_id))
    return match.group(1).replace('_', ' ') if match else str(race_id)

pit_data['year'] = pit_data['raceId'].apply(extract_year)
pit_data['race_name'] = pit_data['raceId'].apply(extract_race_name)

valid_data = pit_data[
    (pit_data['year'] >= 1994) & 
    (pit_data['year'] <= 2022) & 
    (pit_data['pit_sec'].notna()) & 
    (pit_data['positions_gained'].notna())
].copy()

stats = {
    'total_records': len(valid_data),
    'pit_time_min': valid_data['pit_sec'].min(),
    'pit_time_max': valid_data['pit_sec'].max(),
    'pit_time_mean': valid_data['pit_sec'].mean(),
    'pos_change_min': valid_data['positions_gained'].min(),
    'pos_change_max': valid_data['positions_gained'].max(),
    'pos_change_mean': valid_data['positions_gained'].mean()
}

fig_combined = go.Figure()

years = sorted(valid_data['year'].unique())

fig_combined.add_trace(
    go.Scatter(
        x=valid_data['pit_sec'],
        y=valid_data['positions_gained'],
        mode='markers',
        name='Scatter Alle jaren',
        marker=dict(
            size=4,
            opacity=0.6,
            line=dict(width=0.5, color='white'),
            color=valid_data['year'],
            colorscale='Reds',
            cmin=years[0],
            cmax=years[-1],
            colorbar=dict(title="Jaar")
        ),
        customdata=valid_data[['driverId', 'race_name', 'start_pos', 'end_pos', 'year']].values,
        hovertemplate=(
            '<b>%{customdata[0]}</b><br>' +
            'Race: %{customdata[1]}<br>' +
            'Start pos: %{customdata[2]}<br>' +
            'End pos: %{customdata[3]}<br>' +
            'Pittijd: %{x:.2f}s<br>' +
            'Pos. verandering: %{y}<br>' +
            'Jaar: %{customdata[4]}<br>' +
            '<extra></extra>'
        ),
        visible=True,
        meta={'plot_type': 'scatter', 'year': 'all'}
    )
)

fig_combined.add_trace(
    go.Histogram2d(
        x=valid_data['pit_sec'],
        y=valid_data['positions_gained'],
        colorscale='Reds',
        nbinsx=30,
        nbinsy=30,
        name='Density Alle jaren',
        hovertemplate='Pittijd: %{x:.1f}s<br>Positieverandering: %{y}<br>Aantal: %{z}<extra></extra>',
        visible=False,
        meta={'plot_type': 'density', 'year': 'all'}
    )
)

for year in years:
    year_data = valid_data[valid_data['year'] == year]
    
    fig_combined.add_trace(
        go.Scatter(
            x=year_data['pit_sec'],
            y=year_data['positions_gained'],
            mode='markers',
            name=f'Scatter {year}',
            marker=dict(
                size=6,
                opacity=0.8,
                line=dict(width=1, color='white'),
                color='#DC143C',
                colorscale='Reds',
                cmin=years[0],
                cmax=years[-1]
            ),
            customdata=year_data[['driverId', 'race_name', 'start_pos', 'end_pos']].values,
            hovertemplate=(
                '<b>%{customdata[0]}</b><br>' +
                'Race: %{customdata[1]}<br>' +
                'Start pos: %{customdata[2]}<br>' +
                'End pos: %{customdata[3]}<br>' +
                'Pittijd: %{x:.2f}s<br>' +
                'Pos. verandering: %{y}<br>' +
                f'Jaar: {year}<br>' +
                '<extra></extra>'
            ),
            visible=False,
            meta={'plot_type': 'scatter', 'year': year}
        )
    )
    
    fig_combined.add_trace(
        go.Histogram2d(
            x=year_data['pit_sec'],
            y=year_data['positions_gained'],
            colorscale='Reds',
            nbinsx=25,
            nbinsy=25,
            name=f'Density {year}',
            hovertemplate='Pittijd: %{x:.1f}s<br>Positieverandering: %{y}<br>Aantal: %{z}<extra></extra>',
            visible=False,
            meta={'plot_type': 'density', 'year': year}
        )
    )

fig_combined.add_hline(y=0, line_dash="dash", line_color="red", 
                      annotation_text="Geen positieverandering")

slider_steps = []

scatter_visibility_all = []
for trace in fig_combined.data:
    if hasattr(trace, 'meta') and trace.meta:
        scatter_visibility_all.append(trace.meta['plot_type'] == 'scatter' and trace.meta['year'] == 'all')
    else:
        scatter_visibility_all.append(True)

slider_steps.append({
    'args': [
        {'visible': scatter_visibility_all},
        {'title': 'F1 Pittijd vs Positieverandering - Alle jaren (1994-2022) (Scatter Plot)'}
    ],
    'label': 'Alle jaren',
    'method': 'update'
})

for i, year in enumerate(years):
    scatter_visibility = []
    for trace in fig_combined.data:
        if hasattr(trace, 'meta') and trace.meta:
            scatter_visibility.append(trace.meta['plot_type'] == 'scatter' and trace.meta['year'] == year)
        else:
            scatter_visibility.append(True)
    
    slider_steps.append({
        'args': [
            {'visible': scatter_visibility},
            {'title': f'F1 Pittijd vs Positieverandering - {year} (Scatter Plot)'}
        ],
        'label': str(year),
        'method': 'update'
    })

def create_plot_type_buttons():
    return [
        {
            'label': 'Scatter Plot',
            'method': 'update',
            'args': [
                {
                    'visible': [
                        trace.meta['plot_type'] == 'scatter' and trace.meta['year'] == 'all'
                        if hasattr(trace, 'meta') and trace.meta else True
                        for trace in fig_combined.data
                    ]
                },
                {'title': 'F1 Pittijd vs Positieverandering - Alle jaren (1994-2022) (Scatter Plot)'}
            ]
        },
        {
            'label': 'Density Plot',
            'method': 'update',
            'args': [
                {
                    'visible': [
                        trace.meta['plot_type'] == 'density' and trace.meta['year'] == 'all'
                        if hasattr(trace, 'meta') and trace.meta else True
                        for trace in fig_combined.data
                    ]
                },
                {'title': 'F1 Pittijd vs Positieverandering - Alle jaren (1994-2022) (Density Plot)'}
            ]
        }
    ]

fig_combined.update_layout(
    title={
        'text': 'F1 Pittijd vs Positieverandering - Alle jaren (1994-2022) (Scatter Plot)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    xaxis_title='Pittijd (seconden)',
    yaxis_title='Positieverandering',
    font=dict(size=11),
    width=800,
    height=500,
    showlegend=False,
    margin=dict(l=60, r=60, t=70, b=60),
    updatemenus=[
        {
            'buttons': create_plot_type_buttons(),
            'direction': 'down',
            'showactive': True,
            'x': 0.02,
            'xanchor': 'left',
            'y': 0.98,
            'yanchor': 'top',
            'bgcolor': 'white',
            'bordercolor': 'black',
            'borderwidth': 1
        }
    ],
    sliders=[{
        'active': 0,
        'currentvalue': {"prefix": "Selectie: "},
        'len': 0.8,
        'x': 0.1,
        'y': 0,
        'steps': slider_steps
    }]
)
fig_combined.show()